In [6]:
import cv2
import numpy as np
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
app = FaceAnalysis(name='/home/advance/.insightface/models/buffalo_l')
app.prepare(ctx_id=0, det_size=(640, 640))
img = ins_get_image('/home/advance/dev/insightface/2p')
faces = app.get(img)
# print("face box:", faces)
# rimg = app.draw_on(img, faces)
# cv2.imwrite("./t1_output.jpg", rimg)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/advance/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/advance/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/advance/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/advance/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/advance/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

In [7]:
bboxes = []
for face in faces:
    bboxes.append(face["bbox"])

# 重新调整框的坐标
def readjustment_bbox(bboxes, scale = 0.35):
    # print(bboxs)
    re_bboxes = []
    for bbox in bboxes:
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]  
        # 对框的大小按照比例进行缩放
        
        # 框的中心点（x0,y0）    
        x0 = (bbox[0]+bbox[2])/2
        y0 = (bbox[1]+bbox[3])/2

        x1 = x0-(w/2)*(1+scale)
        y1 = y0-(h/2)*(1+scale)
        x2 = x0+(w/2)*(1+scale) 
        y2 = y0+(h/2)*(1+scale)

        # 判断是否越界
        if x1<0:
            x1 = 0
        if y1<0:
            y1 = 0

        if x2>img.shape[1]:
            x2 = img.shape[1]
        if y2>img.shape[0]:
            y2 = img.shape[0]
        re_bboxes.append([x1, y1, x2, y2])

    return re_bboxes

re_bboxes = readjustment_bbox(bboxes)
re_bboxes


[[415.2529724121094, 41.32658309936522, 662.2373840332032, 364.11711196899415],
 [109.26795272827148,
  143.17405471801757,
  345.92446060180663,
  426.5153984069824]]

### 正方形框

In [9]:
# 正方形框

re_bboxes
# 根据对角坐标对人脸进行裁剪
def crop_face(img, bbox):
    """
    Crop face from image given bounding box
    """
    x1, y1, x2, y2 = (int(i) for i in bbox)
    w = x2 - x1
    h = y2 - y1

    # 将框的形状调整为正方形
    if w%2!=0:
        x2 = x2-1
    if h%2!=0:
        y2 = y2-1
    # 计算框的中心点
    x0 = (x1 + x2) / 2
    y0 = (y1 + y2) / 2
    center = (x0, y0)
    # 重新计算宽高
    w = x2 - x1
    h = y2 - y1
    if w>h:
        size = (w, w)
    else:
        size = (h, h)

    # 计算正方形框的左上角和右下角坐标
    # 左上角
    x1 = int(center[0] - size[0] / 2)
    y1 = int(center[1] - size[1] / 2)
    # 右下角
    x2 = int(center[0] + size[0] / 2)
    y2 = int(center[1] + size[1] / 2)
    # 判断是否越界
    if x1<0:
        x1 = 0
    if y1<0:
        y1 = 0

    if x2>img.shape[1]:
        x2 = img.shape[1]
    if y2>img.shape[0]:
        y2 = img.shape[0]
        
    # 重新计算框的中心点
    x0 = (x1 + x2) / 2
    y0 = (y1 + y2) / 2
    center = (x0, y0)

    if y1<=0 or y2>=img.shape[0] or x1<=0 or x2>=img.shape[1]:
        mark = None
        # 创建一个全黑的图像
        face_image = np.zeros((size[1], size[0], 3), dtype=np.uint8)
        # 左边的框
        # 左上角
        if y1<=0:
            if w>h:
                face_image[w-h:, :, :] = img[y1:y2, x1:x2, :]
                # NOTE did那边做完之后按照这个格式：face_image[w-h:, :, :]，进行剪裁
                mark = 1
        
        if x1<=0:
            if h>w:
                face_image[:, h-w:, :] = img[y1:y2, x1:x2, :]
                mark = 2
        # 左下角
        if y2>=img.shape[0]:
            if w>h:
                face_image[:h, :, :] = img[y1:y2, x1:x2, :]
                mark = 3
        if x1<=0:
            if h>w:
                face_image[:, :w, :] = img[y1:y2, x1:x2, :]
                mark = 4

        # 右边的框
        # 右上角
        if y1<=0:
            if w>h:
                face_image[w-h:, :, :] = img[y1:y2, x1:x2, :]
                mark = 5
        if x2>=img.shape[1]:
            if h>w:
                face_image[:, :w, :] = img[y1:y2, x1:x2, :]
                mark = 6
        # 右下角
        if y2>=img.shape[0]:
            if w>h:
                face_image[:h, :, :] = img[y1:y2, x1:x2, :]
                mark = 7
        if x2>=img.shape[1]:
            if h>w:
                face_image[:, :w, :] = img[y1:y2, x1:x2, :]
                mark = 8
    else:
        face_image = img[y1:y2, x1:x2]
        mark = 0
    
    return face_image, size, center, mark

i = 0
for re_bbox in re_bboxes:
    
    face_image, size, center, mark = crop_face(img, re_bbox)
    # 保存裁剪后的图像
    cv2.imwrite(f'cropped_face_{i}.jpg', face_image) 
    i += 1
    print(face_image.shape, size, center, mark)



(322, 322, 3) (322, 322) (538.0, 202.0) 0
(282, 282, 3) (282, 282) (227.0, 284.0) 0


###  获取两个框之间的等距离线进行裁剪

In [ ]:
# 计算等距离线
x12 = re_bboxs[0][1][0]
x21 = re_bboxs[1][0][0]
x_mid = (x12 + x21) / 2


In [ ]:
face_ = []
face = faces[0]["bbox"]


w = face[2] - face[0]
h = face[3] - face[1]


# 对框的大小按照比例进行缩放
scale = 0.35
# 框的中心点（x0,y0）
x0 = (face[0]+face[2])/2
y0 = (face[1]+face[3])/2

x1 = x0-(w/2)*(1+scale)
y1 = y0-(h/2)*(1+scale)
x2 = x0+(w/2)*(1+scale) 
y2 = y0+(h/2)*(1+scale)

# 判断是否越界
if x1<0:
    x1 = 0
if y1<0:
    y1 = 0

if x2>img.shape[1]:
    x2 = img.shape[1]
if y2>img.shape[0]:
    y2 = img.shape[0]


face_  = [x1, y1, x2, y2]


print(face_)

In [ ]:
# 根据对角坐标对人脸进行裁剪
import cv2
import numpy as np  


def crop_face(img, bbox):
    """
    Crop face from image given bounding box
    """
    x1, y1, x2, y2 = (int(i) for i in bbox)
    w = x2 - x1
    h = y2 - y1

    # 将框的形状调整为正方形
    if w%2!=0:
        x2 = x2-1
    if h%2!=0:
        y2 = y2-1
    
    # 重新计算宽高
    w = x2 - x1
    h = y2 - y1

    if w>h:
        h = w
    else:
        w = h
    
    # 计算框的中心点
    x0 = (x1 + x2) / 2
    y0 = (y1 + y2) / 2

    face_image = img[y1:y2, x1:x2]
    return face_image, (x0, y0), (w, h)

bbox = face_
face_image, center, size = crop_face(img, bbox)
print(center)


# 保存裁剪后的图像
cv2.imwrite('cropped_face.jpg', face_image)

In [ ]:
face_image.shape

In [ ]:
size